In [ ]:
# MILE 6. MOUNT THE S3 BUCKET TO THE DATABRICKS ACCOUNT.

In [ ]:
# Import libraries

# pyspark functions
from pyspark.sql.functions import *
# URL processing
import urllib

# Reading the table containing the AWS keys to Databricks
delta_table_path = "dbfs:/user/hive/warehouse/authentication_credentials"

# Read the Delta table to a Spark DataFrame
aws_keys_df = spark.read.format("delta").load(delta_table_path)

# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

# Now the S3 bucket can be mounted by passing in the S3 URL and the desired mount name to dbutils.fs.mount()

# AWS S3 bucket name
AWS_S3_BUCKET = "user-0e172e8c4bc3-bucket"
# Mount name for the bucket
MOUNT_NAME = "/mnt/mount_S3_PDP_mile6"
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

# To check if the S3 bucket was mounted succesfully this command was run: 
# display(dbutils.fs.ls("/mnt/mount_S3_PDP_mile6/topics/0e172e8c4bc3.pin/partition=0/")). 

# Not displaying here to save time on scroling for the evaluator.

In [ ]:
# dbutils.fs.unmount("/mnt/mount_S3_PDP_mile6") - for unmounting

In [ ]:
%sql
-- Disable format checks during the reading of Delta tables
SET spark.databricks.delta.formatCheck.enabled=false

key,value
spark.databricks.delta.formatCheck.enabled,false


In [ ]:
# Define a function to read in JSONs from mounted S3 bucket topic 0e172e8c4bc3.<your-topic-ending>. Replace the topic ending with 'pin', 'geo' or 'user' when calling this function. Function returns a DataFrame.

def spark_read_from_s3(topic_ending):
    # Asterisk(*) indicates reading all the content of the specified file that have .json extension
    file_location = f"/mnt/mount_S3_PDP_mile6/topics/0e172e8c4bc3.{topic_ending}/partition=0/*.json"
    file_type = "json"
    # Ask Spark to infer the schema
    infer_schema = "true"
    # Read in JSONs from mounted S3 bucket chosen topic
    df = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .load(file_location)
    return df

# Calling function to extract all three df's.
df_pin = spark_read_from_s3('pin')
df_geo = spark_read_from_s3('geo')
df_user = spark_read_from_s3('user')


In [ ]:
# Drop the duplicates 
df_pin = df_pin.dropDuplicates()
df_geo = df_geo.dropDuplicates()
df_user = df_user.dropDuplicates()

# Check if duplicates were deleted. Compare to original DF number of rows: 2094.
df_pin.count() 

Out[249]: 1746

In [ ]:
%run Users/kasalotas@yahoo.com/transformation_functions 

In [ ]:
# MILE 7. TASK 1. Data transformation on df_pin dataframe.

df_pin = clean_pin(df_pin)
df_p = df_pin.limit(10)    # Use .limit() to have just 10 rows to display and not the whole df.
df_pin.printSchema()
display(df_p)

root
-- ind: integer (nullable = true)
-- unique_id: string (nullable = true)
-- title: string (nullable = true)
-- description: string (nullable = true)
-- follower_count: integer (nullable = true)
-- poster_name: string (nullable = true)
-- tag_list: string (nullable = true)
-- is_image_or_video: string (nullable = true)
-- image_src: string (nullable = true)
-- save_location: string (nullable = true)
-- category: string (nullable = true)

ind,unique_id,title,description,follower_count,poster_name,tag_list,is_image_or_video,image_src,save_location,category
7028,cc96f759-8aea-4036-8d45-ce881562b7ed,Men's Fashion on Instagram: “#fashionmen #model #2015 #mystylish #fashion #instafashion #body #outfit #nyc #mens #adidas #nike”,"490 Likes, 22 Comments - Men's Fashion (@fashionformen_) on Instagram: “#fashionmen #model #2015 #mystylish #fashion #instafashion #body #outfit #nyc #mens #adidas #nike”",4,Erik Dunteman,"Mode Masculine,Fashion Night,Urban Fashion,Mode Outfits,Fashion Outfits,Fashion Fashion,Fashion Lookbook,Street Fashion,Trendy Outfits",image,https://i.pinimg.com/originals/e4/f1/1b/e4f11bf7c762d766514e0a7b8565b762.jpg,/data/mens-fashion,mens-fashion
3351,282a8cf2-d4f3-4d21-837c-90c953b737cf,Easy Christmas Crafts for Kids - Fruit Loop Candy Cane,"If you're looking for easy Christmas crafts for kids, this is it! You only need 2 things to make them - Fruit Loop cereal and pipe cleaners! They're a great fine motor activity…",29000,Lindsey Blogs,"Kids Food Crafts,Preschool Christmas Crafts,Christmas Crafts For Kids To Make,Christmas Activities For Kids,Christmas Tree Crafts,Toddler Christmas,Toddler Crafts,Preschool Winter,Winter Activities",image,https://i.pinimg.com/originals/f4/40/aa/f440aae805984076e0886407677e0ec2.png,/data/diy-and-crafts,diy-and-crafts
2780,e8bd0e91-9560-42cd-86c4-b754870a0e89,Quick & Easy Kids Halloween Crafts & Activities - Over the Big Moon,These Kids Halloween Crafts and Activities are perfect for Halloween and for kids of all ages - some require craft suppies and others are printable!,79000,Pam from Over the Big Moon,"Fall Crafts For Kids,Toddler Crafts,Kids Crafts,Arts And Crafts,Halloween Crafts For Kindergarten,Halloween Crafts For Kids To Make,Halloween Crafts For Preschoolers,Preschool Halloween Crafts,Winter Craft",image,https://i.pinimg.com/originals/f4/28/50/f428509f9ef819a04f6d2f8224233c22.jpg,/data/diy-and-crafts,diy-and-crafts
7914,37921664-2eeb-438a-bbe4-518847565874,"5 Forgiveness Quotes To Help You Set Yourself Free, Let Go, and Heal - Strong with Purpose | Healing & Intuitive Living","The beauty is that we have the ability to heal and forgive. Read these 5 Forgiveness Quotes To Help You Set Yourself Free, Let Go, and Heal. #quoteoftheday #inspiring",8000,Strong with Purpose | Healing & Intuitive Living,"Quotable Quotes,Wisdom Quotes,True Quotes,Great Quotes,Words Quotes,Motivational Quotes,Inspirational Quotes,Sayings,Quotes Quotes",image,https://i.pinimg.com/originals/8b/95/56/8b95568cda89362b6e31c80c2ea6599e.jpg,/data/quotes,quotes
3008,649ca299-a339-4136-a126-501515c93c8d,Upcycled Denim Rag Rug DIY Instructions,Turn those old jeans into practical rag rugs with just some scrap yarn and a little crochet know-how. I'll show you how to cut the jeans into strips and crochet them together to…,100000,"My Poppet - A Craft, Travel & Food Blog for Colourful Living","Pot Mason Diy,Mason Jar Crafts,Bottle Crafts,Rag Rug Diy,Diy Crochet Rag Rug,Scrap Yarn Crochet,Diy Rugs,Crochet Rug Patterns,Free Crochet",video,https://i.pinimg.com/videos/thumbnails/originals/0b/10/af/0b10af74b987322a6f4384470bf4eaf0.0000001.jpg,/data/diy-and-crafts,diy-and-crafts
4821,bcc09577-7104-4270-8388-f94bef985211,"Cricut Maker Adaptive QuickSwap Tools - Happily Ever After, Etc.",This is a sponsored conversation written by me on behalf of Cricut. The opinions and text are all mine. Earlier this year when Cricut released the Scoring Wheel I took a look at…,53000,Happily Ever After Etc,"Homemade Wedding Invitations,Affordable Wedding Invitations,Cricut Wedding Invitations,Wedding Stationery,Invitation Paper,Wedding Planner,Making Ideas,Event Planning,Super Easy",image,https://i.pinimg.com/originals/1f/3c/7d/1f3c7da004601c12f4747ca6594da832.jpg,/data/event-planning,event-planning
2098,e7c36d58-9fd6-457b-b093-390f1e6e2e62,"How to Make Easy DIY Gnomes | Free Pattern Download PDF, Ideas & Instructions | Be Brave and Bloom","How to make easy DIY gnomes. FREE P

In [ ]:
# MILE 7. TASK 2. Data transformation on df_geo dataframe. 

df_geo = clean_geo(df_geo)
df_g = df_geo.limit(10)
df_geo.printSchema()
display(df_g)


root
-- ind: long (nullable = true)
-- country: string (nullable = true)
-- coordinates: string (nullable = true)
-- timestamp: timestamp (nullable = true)

ind,country,coordinates,timestamp
813,French Southern Territories,"-70.9146, -136.554",2022-06-13T23:08:43.000+0000
8677,Bouvet Island (Bouvetoya),"-84.3984, -144.933",2020-02-12T23:24:01.000+0000
6547,Falkland Islands (Malvinas),"-51.4908, -55.086",2021-11-25T15:37:05.000+0000
204,United States Virgin Islands,"29.2096, -13.1258",2018-11-23T15:01:11.000+0000
5379,Falkland Islands (Malvinas),"47.3574, -88.0068",2018-08-21T01:22:46.000+0000
974,Svalbard & Jan Mayen Islands,"-56.9556, 94.6183",2022-03-17T09:24:07.000+0000
5244,French Southern Territories,"27.8614, -130.864",2021-01-26T03:26:43.000+0000
6277,Falkland Islands (Malvinas),"-51.4908, -55.086",2019-02-21T03:07:17.000+0000
303,United States Virgin Islands,"31.1892, -163.128",2022-08-17T16:18:21.000+0000
10369,Bouvet Island (Bouvetoya),"-79.0994, -101.415",2020-02-01T10:43:06.000+0000


In [ ]:
# MILE 7. TASK 3. Data transformation on df_user dataframe.

df_user = clean_user(df_user)
df_u = df_user.limit(10)
df_user.printSchema()
display(df_u)

root
-- ind: long (nullable = true)
-- user_name: string (nullable = true)
-- age: long (nullable = true)
-- date_joined: timestamp (nullable = true)

ind,user_name,age,date_joined
1487,Jonathan Rodriguez,30,2016-09-05T12:31:44.000+0000
6806,Christopher Daniel,25,2016-04-15T17:06:49.000+0000
4761,Christopher Norman,40,2017-03-07T21:33:30.000+0000
10721,Garrett Hernandez,53,2017-04-05T13:59:02.000+0000
10675,Christopher Combs,42,2016-02-08T16:47:33.000+0000
2532,Danielle Hernandez,24,2016-03-24T23:39:08.000+0000
7206,Christina Anderson,27,2016-03-03T11:25:21.000+0000
2267,Andrea Blankenship,27,2016-04-11T21:27:52.000+0000
7790,Michelle Gutierrez,39,2017-07-19T07:12:04.000+0000
10600,Robert Mclaughlin,49,2016-02-27T04:42:03.000+0000


In [ ]:
# MILE 7. TASK 4. Category with the most pins by country.

In [ ]:
from pyspark.sql.functions import col, row_number 
from pyspark.sql.window import Window 

# Join dataframes df_pin and df_geo to obtain necessary columns in one table
df_pin_geo = df_pin.join(df_geo, df_pin["ind"] == df_geo["ind"], how="inner")

# Group by country and category, then find the max count for each group 
cat_count = df_pin_geo.groupBy("country", "category").agg(count("category").alias("count"))
result = cat_count.groupBy("country", "category").agg(max("count").alias("category_count")) 

# Use window function to get the row_number for each partition (county).
windowSpec = Window.partitionBy("country").orderBy(col("category_count").desc())
final_result = result.withColumn("rank", row_number().over(windowSpec)).filter(col("rank") == 1).drop("rank")

# Show the final result 
display(final_result)

country,category,category_count
Afghanistan,education,14
Albania,art,29
Algeria,quotes,38
American Samoa,tattoos,12
Andorra,tattoos,11
Angola,diy-and-crafts,4
Anguilla,diy-and-crafts,6
Antarctica (the territory South of 60 deg S),tattoos,6
Antigua and Barbuda,art,4
Argentina,tattoos,11


In [ ]:
# MILE 7. TASK 5. The most popular category each year.

In [ ]:
from pyspark.sql.functions import year

# Join dataframes df_pin and df_user to obtain necessary columns in one table
df_pin_user = df_pin.join(df_user, df_pin["ind"] == df_user["ind"], how="inner")

# Filter data between 2016 and 2018 (data reached only max to 2018)
filtered_df = df_pin_user.filter((year(df_pin_user["date_joined"]) >= 2016) & (year(df_pin_user["date_joined"]) <= 2018))

# Extract year from the timestamp column
filtered_df = filtered_df.withColumn("post_year", year(filtered_df["date_joined"]))

# Group by 'category' and 'post_year', count the number of posts
result = filtered_df.groupBy("category", "post_year").count()
result = result.orderBy(col("post_year").desc(), col("count").desc())
result = result.withColumnRenamed("count", "category_count")

# Display the resulting DataFrame
display(result)


category,post_year,category_count
vehicles,2017,31
event-planning,2017,30
finance,2017,28
christmas,2017,27
mens-fashion,2017,26
beauty,2017,21
home-decor,2017,20
education,2017,18
travel,2017,17
art,2017,16


In [ ]:
# MILE 7. TASK 6. User with the most fallowers in each country.

In [ ]:
# STEP 1.
# Group by country and category, then find the max count for each group 
follower_df = df_pin_geo.select("country","poster_name", "follower_count")
max_followers = follower_df.orderBy(col("follower_count").desc())

# STEP 2.
# Find the country with the greatest follower count. Use limit() to return a new DataFrame.
max_followers = max_followers.drop("poster_name")
result = max_followers.limit(1)
display(result)

country,follower_count
Anguilla,15000000


In [ ]:
# MILE 7. TASK 7. The most popular category for different age groups.

In [ ]:
from pyspark.sql.functions import udf

# Use udf function and withColumn to create a new column with age ranges based on "age" column.
age_group = udf(lambda age: '18-24' if age >= 18 and age <= 24 else              # No users <18
                       '25-35' if (age >= 25 and age <= 35) else
                       '36-50' if (age >= 36 and age <= 50) else
                       '50+' if (age > 50) else '')

df_age_group = df_pin_user.withColumn("age_group", age_group(df_pin_user.age))

# Group by age_group and category, then find the max count for each group
category_count = df_age_group.groupBy("age_group", "category").agg(count("category").alias("category_count"))
result = category_count.groupBy("age_group", "category").agg(max("category_count").alias("category_count")) 

# Use window function to get the row_number for each partition (age_group).
windowSpec = Window.partitionBy("age_group").orderBy(col("category_count").desc())
final_result = result.withColumn("rank", row_number().over(windowSpec)).filter(col("rank") == 1).drop("rank")

display(final_result)

age_group,category,category_count
18-24,tattoos,88
25-35,christmas,55
36-50,finance,40
50+,vehicles,21


In [ ]:
# MILE 7. TASK 8. The median follower count for users based on age group.

In [ ]:
# Use previously defined df_age_group to select relevant columns
sorted_followers = df_age_group.select("age_group", "follower_count")

# Group by age_group, then find the median count for each group 
sorted_followers = sorted_followers.orderBy(col("age_group").asc())
median_follower_count = sorted_followers.groupBy("age_group").agg(expr("percentile_approx(follower_count, 0.5) as median"))
display(median_follower_count)


age_group,median
18-24,127000
25-35,22000
36-50,7000
50+,1000


In [ ]:
# MILE 7. TASK 9.

In [ ]:
# Filter data between 2015 and 2020 
filtered_df = df_pin_user.filter((year(df_pin_user["date_joined"]) >= 2015) & (year(df_pin_user["date_joined"]) <= 2020))

# Extract year from the timestamp column
post_year_df = filtered_df.withColumn("post_year", year(filtered_df["date_joined"]))

# Group by 'post_year', count the number of unique poster_names assuming that poster_name is unique to each account
result = post_year_df.groupBy("post_year").agg(countDistinct("poster_name").alias("number_users_joined"))

# Display the resulting DataFrame
display(result)


post_year,number_users_joined
2015,300
2016,675
2017,275


In [ ]:
# MILE 7. TASK 10.

In [ ]:
# Selec columns from previously defined post_year_df, which fltered df_pin_user based on year
follower_df = post_year_df.select("post_year", "follower_count")

# Group by age_group, then find the median count for each group 
median_follower_count = follower_df.groupBy("post_year").agg(expr("percentile_approx(follower_count, 0.5) as median_follower_count"))
display(median_follower_count)

post_year,median_follower_count
2015,150000
2016,19000
2017,3000


In [ ]:
# MILE 7. TASK 11.

In [ ]:
# Add a age_group column into previously defined post_year_df (joint pin data and user data df with post_year column)
df_post_year_age_group = post_year_df.withColumn("age_group", age_group(post_year_df.age))

# Select columns that should be displayed
follower_df = df_post_year_age_group.select("post_year", "age_group", "follower_count")

# Group by age_group, then find the median count for each group 
follower_df = follower_df.orderBy(col("post_year").desc(), col("age_group").asc())
median_follower_count = follower_df.groupBy("post_year", "age_group").agg(expr("percentile_approx(follower_count, 0.5) as median_follower_count"))

display(median_follower_count)

post_year,age_group,median_follower_count
2017,18-24,12000
2017,25-35,2000
2017,36-50,3000
2017,50+,1000
2016,18-24,46000
2016,25-35,21000
2016,36-50,8000
2016,50+,1000
2015,18-24,267000
2015,25-35,42000
